# Using Customized and External Cell and Channel Models in BioNet

Through the previous tutorials, we've been using networks that are built off of cell models that are downloaded from the [Allen Cell Types Database](https://celltypes.brain-map.org/data). While the Allen Cell Types Databases has a wide and continuing growing selection of optimized cell-type models to download and use, for many scientists, it may not meet their needs. BMTK supports the ability to incorporate many different types of cell and synaptic models into their simulations, including pre-built models from non-Allen Institute organizations, and allows users multiple ways to create their own customized models. 

This section will focus on single-cell spiking models, including conductance-based models with multiple channels and/or compartments per model. We do not recommend using Rates-based models with BioNet, and they will likely not work.

In the following section, we will show users multiple ways they can import or create their own networks and simulations. Many of the multi-compartment biophysically detailed models developed for the NEURON simulator are written in [.hoc](https://www.neuron.yale.edu/neuron/static/new_doc/programming/hocsyntax.html) format. Another popular format is [NeuroML v2](https://docs.neuroml.org/Userdocs/NeuroMLv2.html). In other cases, models may be written in Python. BioNet supports these options and gives users the ability to readily implement other options, too.

If you are unfamiliar with how and where to find cell and channel models that have been created by other scientists, two good options to start looking at are:
1. [ModelDB](https://modeldb.science/)
2. [Open Source Brain](https://v2.opensourcebrain.org/)


---

## Contents
1. [Example: Loading a simple NEURON HOC template cell model into your network simulation](#example-simple-hoc)
2. [Example: Overriding the way which BMTK creates cell models before simulation](#example-modify-init)
3. [Example: Loading a python-based neuronal model into your simulation](#example-python-models)
4. [Example: Building your own python-based cell models](#example-custom-cells)
5. [Example: Adding customized mechanisms into an existing cell model](#model-processing-added-mech)
6. [Example: Modifing channel conductance in an instaniated cell](#model-processing-channel-conductance)

---

## Cell Models
---

### Example: Loading in a Simple Hoc Model <a class="anchor" id="example-simple-hoc">


##### Step 1: 

**Note**: *This step has already been done in the current notebook and does not need to be repeated, but may be useful when initializing other examples*

The first thing we'll do is to download and unpack the model files from ModelDB, which for the Hay L5PC models can be found at  [https://modeldb.science/139653](https://modeldb.science/139653). At the site, you can use the download button to download a zip file with the appropriate .hoc (cell model), .mod (ion channels), and .asc (morphologies) files.


<br>
<div>
  <img src="../images/modeldb_hay_screen.png" width="900" align="center" style="margin-left:5px"/>    
</div>
<br clear="left">

To run properly, we will need to move the following files from the download zip file into our **components/** folder so BMTK will be able to find them during the simulation:
* Move **mod/** folder => **componets/mechanisms/mod/**.
* Move **morphologies/\*.asc** files => **components/morphologies/** folder.
* Move **models/L5PCtemplate.hoc** and **models/L5PCbiophys4.hoc** => **componets/templates/** folder (*note*: The zip file contains four different version of L5PCbiophys model stored in different hoc files. However, only one can be loaded into NEURON at a time, hence we are only moving one of the files and not all of them).
 


The model contains customized ion channels in the **components/mechanisms/mod/** folder that require compiling using the `nrnivmod` command. In a shell or using jupyter notebook you'll need to run the following:

In [1]:
! cd components/mechanisms && nrnivmodl modfiles

/local1/.local/miniconda-23.10.0/bin:/local1/.local/miniconda-23.10.0/condabin:/local1/.local/miniconda-23.10.0/bin:/local1/apps/pycharm/bin:/usr/local/bin:/usr/local/sbin:/usr/lib64/qt-3.3/bin:/home/kaeld/perl5/bin:/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/bin:/sbin:/home/kaeld/.rvm/bin:/home/kaeld/.rvm/bin:/usr/pgsql-9.4/bin:/home/kaeld/bin


##### Step 2: Building the Network

Building a network using the downloaded L5PC Hay model is very similar to how we built biophysically detailed network models in [previous chapters](../Ch3_multicells/3.%20Multipopulation%20biophysical.ipynb). We use the `add_nodes()` method to add a population of 1 or more L5PC `biophysical` cells and the `add_edges()` to connect them up with other cells. However, due to the model type and parameters, we will need to make some changes compared to how we utilize Allen-Cell Types Database models.

* We set **model_template**=`hoc:L5PCtemplate` instead of the Allen `ctdb:Biophys1` value. The name of the Hoc Template for loading this particular model comes from the components/templates/L5PCtemplate.hoc file
* In our Allen Cell-Types models, we had set the value **model_processing** to 'aibs_perisomatic', which told bmtk to call the special post-initialization directive `aibs_perisomatic`, which would modify each cell object in a specific manner. Although it is possible to create custom directives to update and modify each cell after the template has been loaded (see below), we don't want to do so. Instead, we leave the model_processing attribute blank.
* In our Allen Cell-Types models, we also had a **dynamics_params** option that would update and add cell parameters after the cell had been initialized. However, for our L5PC Hay model, we have no current need to update the parameters (we'll instead use the parameter values stored in the .hoc files), so we will not have any dynamics_params attributes.
    

In [2]:
from bmtk.builder.networks import NetworkBuilder

net = NetworkBuilder('L5')
net.add_nodes(
    N=10,
    model_type='biophysical',
    model_template='hoc:L5PCtemplate',
    morphology='cell3.asc'
)
net.build()
net.save(output_dir='network_L5')

In [3]:
virt_exc = NetworkBuilder('virt_exc')
virt_exc.add_nodes(
    N=20,
    model_type='virtual',
    ei_type='exc'
)
conns = virt_exc.add_edges(
    source=virt_exc.nodes(),
    target=net.nodes(),
    connection_rule=12,
    model_template='Exp2Syn',
    dynamics_params='AMPA_ExcToExc.json',
    distance_range=[0.0, 1.0e20],
    target_sections=['soma', 'basal', 'apical'],
    delay=2.0,
    syn_weight=0.01
)

virt_exc.build()
virt_exc.save(output_dir='network_L5')

##### Step 3: Running the simulation

Running the simulation with the downloaded L5PC models is no different than running the simulations with the Allen Cell-Types Database cell models. BMTK will be able to use the configuration **components** sections along with the **model_template** and **morphology** values to automatically load in our 10 L5PC cells. The results for these cells, including spikes, Vm traces, etc., will be saved in the same format and paths as our other examples.

In [4]:
from bmtk.simulator import bionet

bionet.reset()
conf = bionet.Config.from_json('config.L5.json')
conf.build_env()

net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

numprocs=1
2024-04-30 17:00:56,655 [INFO] Created log file


INFO:NEURONIOUtils:Created log file


2024-04-30 17:00:56,763 [INFO] Building cells.


INFO:NEURONIOUtils:Building cells.


2024-04-30 17:01:09,737 [INFO] Building recurrent connections


...

INFO:NEURONIOUtils:    step:15000 t_sim:1500.00 ms


2024-04-30 17:03:55,122 [INFO]     step:20000 t_sim:2000.00 ms


INFO:NEURONIOUtils:    step:20000 t_sim:2000.00 ms


2024-04-30 17:03:55,141 [INFO] Simulation completed in 2.0 minutes, 45.21 seconds 


INFO:NEURONIOUtils:Simulation completed in 2.0 minutes, 45.21 seconds 


### Example: Modifying Model Initialization <a class="anchor" id="example-modify-init">

If we search Open Source Brain, we can see other own more recent versions of [L5 Pyramidal Cell models from the same lab](https://v2.opensourcebrain.org/repositories/2180). Suppose we wanted to incorporate these newer models alongside the original L5 PC network created above. We would need to do the following:
1. Download the [files](https://github.com/OpenSourceBrain/267587/tree/master/Single%20Cell%20Modelling/Current-Step%20Simulations/L5Pyr_young)
2. Move the morphology, mode, and model template files to their appropriate location
3. Since these files also include new .mod files, we would have to recompile them:
```bash
  $ cd components/mechanisms && nrnivmodl modfiles
```

The newer models have a different morphology file (`Hl5PN1.swc`) and HOC Template name (`HNTemplate`). So in order to add these new cells to the old one we can add the following call in our build script (**build_network.L5_updated.py**):

```python
net.add_nodes(
    N=5,
    model_type='biophysical',
    model_template='hoc:HNTemplate',
    morphology='HL5PN1.swc'
)
```

*Note: We'll also modify the script to write the network to a different directory*


We could try to run the **L5_updated** model like before, but it would throw an error

<div class="alert alert-block alert-info">
⚠️ NEURON: init not enough arguments
</div>

The Reason: **We are not calling the HNTemplate correctly.**

If we look through the code and the documentation, we can see that, to instantiate an HNTemplate cell model instance, we must pass two parameters: The morphology file and the cell-type (one of a preset number of string values that will determine how specific properties are set within the model). In this case, the call would look like:

```python
  h.HNTemplate(morphology_file_path, "HL5PN1")
```

The built-in BMTK function that tries to load hoc files, [loadHOC](https://github.com/AllenInstitute/bmtk/blob/develop/bmtk/simulator/bionet/default_setters/cell_models.py#L43), does not know how to load this specific template. And even if it did, you may run into other models that require different, unseen ways of initialization. 

While it is possible to overwrite the `loadHOC` function, it is not ideal. To avoid breaking existing models, we would need to have multiple cases based on each possible template.

**Solution**

Instead of using one function to load in every template, we can create our own custom function to load the HNTemplate. To achieve this, BMTK uses a special [python decorator](https://peps.python.org/pep-0318/) to assign to function users have written (in either the notebook or run_bionet.py script). Below, we show an example of how to do this using a custom function we'll call loadHNTemplate():

In [4]:
from neuron import h
from bmtk.simulator.bionet import cell_model
from bmtk.simulator.bionet.io_tools import io

@cell_model(directive='hoc:HNTemplate', model_type='biophysical')
def loadHNTemplate(cell, template_name, dynamics_params):
    io.log_info(f'Calling loadHNTemplate() for cell {cell["node_id"]}, template {template_name}')
    hobj = h.HNTemplate(cell['morphology'], "HL5PN1")
    return hobj

 * The `@cell_model` is a decorator we put at the front of our function. Using `directive=hoc:Template` and `model_type=biophysical` parameters indicates to BMTK that whenever a cell model needs to be built for a *biophysical*, *HNTemplate* hoc template, it will use this function.
 * Our function requires three parameters;
   * The `cell` parameter contains attributes of each individual cell and can be used to extract properties like the morphology file, the node-id, coordinates, etc.
   * The `template_name` is, in this case, just the string value "HNTemplate"
   * The `dynamics_params` is a dictionary from the **dynamics_params** json or hdf5. In this network, we never set the dynamics_params attributes, so the value will be None.
 * We need to return either a hoc-object or some other Python class built using the NEURON API.
 * For a sanity check, we also included a log statement. We should only see this message for the 5 newer HNTemplate models, not for the other original "L5PCTemplate" cells.


During simulation, this function will be called for every cell with matching "template_name" and "model_type".

In [5]:
bionet.reset()
conf = bionet.Config.from_json('config.L5_updated.json')
conf.build_env()

net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

2024-04-30 17:06:38,817 [INFO] Created log file


INFO:NEURONIOUtils:Created log file


Mechanisms already loaded from path: ./components/mechanisms.  Aborting.
2024-04-30 17:06:38,863 [INFO] Building cells.


INFO:NEURONIOUtils:Building cells.


2024-04-30 17:06:45,669 [INFO] Calling loadHNTemplate() for cell 5, template HNTemplate


...

INFO:NEURONIOUtils:    step:15000 t_sim:1500.00 ms


2024-04-30 17:11:04,504 [INFO]     step:20000 t_sim:2000.00 ms


INFO:NEURONIOUtils:    step:20000 t_sim:2000.00 ms


2024-04-30 17:11:04,529 [INFO] Simulation completed in 4.0 minutes, 16.81 seconds 


INFO:NEURONIOUtils:Simulation completed in 4.0 minutes, 16.81 seconds 


### Example: Loading a python-based neuronal model into your simulation <a class="anchor" id="example-python-models">

Often, rather than being written in old .hoc format, scientists may instead release their models written in Python. For example, in this [mouse Purkinje cell model](https://github.com/OpenSourceBrain/267694/tree/main/mouse_model), the model is itself saved in the `Purkinje_morph_1.Purkinje_Morph_1` Python class. In nearly the same manner as we did with the .hoc files above, we can import these Python-based single-cell models into our network to use with BMTK.

As before, we must download the necessary files, but in this case, the *.py files, including the file containing the cell-model class/template, should be in the same directory as our `run_bionet.py` script so that we may import it. Also, the Python script expects the .asc morphology file to be hard-coded in the path **./morphology/soma_10c.asc**, so we'll put it there instead of the normal components directory (unless you want to edit the file).


When we build our population of Purkinje cells, the "model_template" will have schema value `python` and a value that is just the name of the resources, e.g., the python class `Purkinje_morph_1`.

```
net.add_nodes( 
    pop_name='Scnn1a',
    model_type='biophysical',
    model_template='python:Purkinje_morph_1',
    spines_on=0
)
```

So that BMTK can load the template, we will once again register a special function using the `cell_model` decorator. The class can be built with or without dendritic spines, which we are required to pass in. The `__init__` method of `Purkinje_Morpho_1` will build the model for each given cell, which we will pass back to BTMK.

Also, for good measure, we added some optional logging.

In [1]:
from bmtk.simulator import bionet
from bmtk.simulator.bionet.io_tools import io
from Purkinje_morpho_1 import Purkinje_Morpho_1

@bionet.cell_model(directive='python:Purkinje_morph_1', model_type='biophysical')
def loadPurkinjeModel(cell, template_name, dynamics_params):
    io.log_info(f'Loading cell {cell.node_id}, template {template_name}, with spines_on={cell["spines_on"]}')
    cell = Purkinje_Morpho_1(cell['spines_on'])
    return cell

In [2]:
bionet.reset()
conf = bionet.Config.from_json('config.purkinje.json')
conf.build_env()

net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

2024-04-30 16:42:04,050 [INFO] Created log file
2024-04-30 16:42:04,174 [INFO] Building cells.
2024-04-30 16:42:04,176 [INFO] Loading cell 0, tempalate Purkinje_morph_1, with spines_on=0

3856 lines read
indiv number 138
2024-04-30 16:42:04,470 [INFO] Loading cell 1, tempalate Purkinje_morph_1, with spines_on=1

3856 lines read
indiv number 138
spine active!!
dend_total_len 7680
list_dend_complete 7680
somma_location_spines 7680
spines_placed 0
spines_placed 500
spines_placed 1000
spines_placed 1500
spines_placed 2000
spines_placed 2500
spines_placed 3000
spines_placed 3500
spines_placed 4000
spines_placed 4500
spines_placed 5000
spines_placed 5500
spines_placed 6000
spines_placed 6500
spines_placed 7000
spines_placed 7500
spines_syn:  spine_head_0
spines_syn:  spine_head_5000
spine_head_aa        between 0 - 0.3     -> 0
spine_head_aa_SC     between 0.3 - 0.75  -> 1494
spine_head_pf_SC     between 0.75 - 1.5  -> 6184
!!!!!!!!!!!!!!!       spine_total 7678        !!!!!!!!!!!!!!!
2024-0

### Example: Building your own python-based cell models <a class="anchor" id="example-custom-cells">

Another option is to build a cell instance yourself using the NEURON Python API. As with above, you just need to change the **model_template** parameter for the given cell(s) you want to build from scratch and create your own `@cell_model` function that will be called prior to the simulation to generate individual NEURON cell instances for appropriate node in the network. The code can be put inside your **run_bionet.py** script or imported from another file in the same directory.

The following example is a simple model taken from the [NEURON ball-and-stick tutorial](https://neuron.yale.edu/neuron/docs/ball-and-stick-model-part-1) (See *build_network.ball_and_stick.py* file to check how the network was created).

In [1]:
from bmtk.simulator import bionet
from bmtk.simulator.bionet.io_tools import io
from neuron import h

class BallAndStick:
    def __init__(self, cell):
        self._gid = cell['node_id']
        self._setup_morphology(cell)
        self._setup_biophysics(cell)

    def _setup_morphology(self, cell):
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)
        self.all = [self.soma, self.dend]
        self.dend.connect(self.soma)
        self.soma.L = self.soma.diam = 12.6157
        self.dend.L = 200
        self.dend.diam = 1
    
    def _setup_biophysics(self, cell):
        for sec in self.all:
            sec.Ra = 100    # Axial resistance in Ohm * cm
            sec.cm = 1      # Membrane capacitance in micro Farads / cm^2
        self.soma.insert('hh')                                          
        for seg in self.soma:
            seg.hh.gnabar = cell['gnabar']  # Sodium conductance in S/cm2
            seg.hh.gkbar = cell['gkbar']  # Potassium conductance in S/cm2
            seg.hh.gl = cell['gl']    # Leak conductance in S/cm2
            seg.hh.el = cell['el']     # Reversal potential in mV
        # Insert passive current in the dendrite                       # <-- NEW
        self.dend.insert('pas')                                        # <-- NEW
        for seg in self.dend:                                          # <-- NEW
            seg.pas.g = cell['g_pas']  # Passive conductance in S/cm2  # <-- NEW
            seg.pas.e = cell['e_pas']    # Leak reversal potential mV  # <-- NEW 
    
    def __repr__(self):
        return 'BallAndStick[{}]'.format(self._gid)

@bionet.cell_model(directive='python:loadBAS', model_type='biophysical')
def loadPurkinjeModel(cell, template_name, dynamics_params):
    bas_cell = BallAndStick(cell)
    io.log_info(f'Adding {bas_cell} to network')
    return bas_cell


In [2]:
bionet.reset()
conf = bionet.Config.from_json('config.ball_and_stick.json')
conf.build_env()

net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

2024-05-01 13:59:49,811 [INFO] Created log file
2024-05-01 13:59:49,948 [INFO] Building cells.
2024-05-01 13:59:49,952 [INFO] Adding BallAndStick[0] to network
2024-05-01 13:59:49,953 [INFO] Adding BallAndStick[1] to network
2024-05-01 13:59:49,955 [INFO] Adding BallAndStick[2] to network
2024-05-01 13:59:49,956 [INFO] Adding BallAndStick[3] to network
2024-05-01 13:59:49,958 [INFO] Adding BallAndStick[4] to network
2024-05-01 13:59:49,959 [INFO] Building recurrent connections
2024-05-01 13:59:49,960 [INFO] Building virtual cell stimulations for virt_exc_spikes
2024-05-01 13:59:50,039 [INFO] Running simulation for 2000.000 ms with the time step 0.100 ms
2024-05-01 13:59:50,040 [INFO] Starting timestep: 0 at t_sim: 0.000 ms
2024-05-01 13:59:50,042 [INFO] Block save every 5000 steps
2024-05-01 13:59:51,006 [INFO]     step:5000 t_sim:500.00 ms
2024-05-01 13:59:51,810 [INFO]     step:10000 t_sim:1000.00 ms
2024-05-01 13:59:52,491 [INFO]     step:15000 t_sim:1500.00 ms
2024-05-01 13:59:53,4

For more information on how to build cells from scratch using NEURON see the [official documentation](https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/topology.html).

## Modifying an existing cell model
---

Very often, modelers may want to use an already existing cell model but make some minor changes of their own. For example, you may want to download a prebuilt channel mechanism (in the form of a .mod or .hoc file) to insert into a subset of cells in an already existing network. You may want to edit the cell morphologies to add or remove sections of the neuron. You may want to adjust the channel densities based on the location of the section.

One option to achieve this is to take an existing Hoc, Python, NeuroML, etc., cell model template and manually edit the file! This is **NOT** a good option for multiple reasons; your changes may lead to unintended side-effects - especially if you're not familiar with the language or how it was created. On top of that, having to rebuild the network and have multiple versions of a Hoc/Python template can be tedious if you are trying to test and debug your changes on the fly.

**Solution**

SONATA and BMTK provide support for making changes to an already instantiated cell model using the [**model_processing**](https://github.com/AllenInstitute/sonata/blob/master/docs/SONATA_DEVELOPER_GUIDE.md#nodes---optional-reserved-attributes) attribute. When an attribute is defined for a subpopulation of cells, it defines a directive and/or function to call on said cell immediately after it has been initialized.

In BMTK, implementing your own custom function for "model_processing" is done in the same manner as with cell-loading. You can use the `@model_processing` decorator to a function that is inside or called in your *run_bionet.py* script.

```python
from bmtk.simulator.bionent import model_processing

@model_processing
def process_my_cell(hoc_obj, cell, dynamics_params):
    updated_hoc_obj = # ... modify hoc object based on parameters stored in cell and/or dynamics params.
    return updated_hoc_obj
```

Then, when building the cells (or by editing either the nodes.h5 or node_types.csv file), you can add a reference to the `process_my_cell` function.

```python
net.add_nodes(
    model_type='biophysical',
    model_template='hoc:Biophys1',
    model_processing='process_my_cell`,
    ...
)
```

BMTK will also allow you to call multiple *model_process* functions in sequence by separating the calls by a comma:

```python
net.add_nodes(
    model_type='biophysical',
    model_template='hoc:Biophys1',
    model_processing='process_cell_1,process_cell_2`,
    ...
)
```

### Example: Adding customized mechanisms into an existing cell model <a class="anchor" id="model-processing-added-mech">

Besides cell models, repositories like ModelDB and Open Source Brain also contain a wide array of membrane density mechanisms (e.g., currents, and channels) that we may want to add to our own existing cell models. In NEURON, these are typically written as .mod files in the NMODL language and require pre-compilation using the *nrnivmodl* command. 

In this example, we will try downloading arbitrary sodium channel models and show how we can dynamically insert them into our L5 cells using the `@model_processing` directive.

#### Step 1: Downloading and compiling mechanisms

First, we will find mechanisms models (e.g., .mod) files to use in our models. Here we [randomly choose sodium channels](https://modeldb.science/140249?tab=1) from ModelDB. Click on the download button to download the files.

We would then need to move the downloaded .mod files into the *components/mechansism/modfiles* and rerun the command `cd components/mechanisms && nrnivmodl`. However, to make things more logical, BMTK allows us to have multiple *mechanisms* directories, so if it turns out in the end that we don't want to use the new mechanisms, they will be easier to delete. Instead, we will put the .mod files into *components/dLGN_mechanisms/modfiles*, and run `nrnivmodl` from the new directory. Then, to make sure BMTK knows where to find the new mechanisms, we need to update the components section of the SONATA config:

```json
"components": {
    "templates_dir": "$COMPONENTS_DIR/templates",
    "mechanisms_dir": [
        "$COMPONENTS_DIR/mechanisms", 
        "$COMPONENTS_DIR/dLGN_mechanisms"
    ],
    ...
}
```

#### Step 2: Setting directive for a population of neurons

We need to make minor updates to our build network so that every subset of cells that will have these new mechanisms added will have model_processing attribute pointing to our customized post-initialization function.

```python
net.add_nodes(
    N=1,
    model_type='biophysical',
    model_template='hoc:L5PCtemplate',
    morphology='cell3.asc',
    model_processing='add_channels'
)
```

Or if you don't want to rebuild the entire network, you can also just add a **model_processing** to the node_types.csv.

#### Step 3: Add model_processing function and run

Now, we can add our customized `add_channels` function, which will be called for each applicable cell right before the simulation. Like with the `@cell_model` directive, the function can be placed inside or imported into our *run_bionet.py* script, has a predefined signature, and should return an HOC instantiated cell object.

Here, we will have our `add_channels` simply iterate through each section of the cell and insert the downloaded compiled IC channel:
```
@bionet.model_processing
def add_channels(hoc_obj, cell, dynamics_params):
    for sec in hoc_obj.all:
        if 'axon' in sec.name():
            continue

        sec.insert('iahp')
        setattr(sec, 'gkbar_iahp', 3.0e-04)
        setattr(sec, 'taum_iahp', 0.5)
        
    return hoc_obj
```
Notes:
* For most models, we can use the `.all` property to iterate through every section of the cell. Depending on the model, you may also have section iterators like `.soma`, `.axon`, `.dend`, `.apical`, `.basal`.
* We don't want to insert IC channels into the axon sections. Unfortunately, there is no flag to indicate if a section is a soma, axon, dend, etc. So instead, we use the section's `name` to select axons (Typically, a section name for a HOC or Python template will be **TemplateName\[cell_id\].axon\[sec_num\]**
* Next, we use the `.insert` command to insert the channel into our section
* We use the Python' setattr' function to change the default values of parameters. The specific mod file will determine which parameters can be changed and their range.

You can run an example using:
```
$ python run.added_channels.py config.added_channels.json
```

### Example: Modifying channel conductance in an instantiated cell.  <a class="anchor" id="model-processing-channel-conductance">

Another thing we can also use the `@model_processing` directive is to update intrinsic properties, or even the morphology and topology, of a given cell right before running a simulation. 

In this example, we will want to take an already existing model and modify the conductance of the hyperpolarization-activated channels (**Ih**) in the apical dendrites based on their distance from the soma. After some investigation, we find that the conductance for our model is controlled by the parameter `gbar_Ih`. So, we want to half this parameter for all sections that are at a certain arc-length distance from the soma. We define `model_processing` function `adjust_densities`.

In [5]:
from bmtk.simulator import bionet
from bmtk.simulator.bionet.io_tools import io

@bionet.model_processing
def adjust_densities(hoc_obj, cell, dynamics_params):
    n_segs = 0
    n_segs_updated = 0
    for sec in hoc_obj.apic:
        for seg in sec:
            n_segs += 1
            if h.distance(seg) > 1000:
                n_segs_updated += 1
                org_cond = getattr(seg, 'gbar_Ih')
                new_cond = org_cond*0.5
                setattr(seg, 'gbar_Ih', new_cond)
                
    io.log_info(f'{n_segs_updated} out of {n_segs} apical dendritic segments updated.')
    return hoc_obj


Some notes to consider about this function:
* To only iterate through the **apic** section types (which will ignore soma, axon, and basal section), we use `hoc_obj.apic` list instead of `hoc_obj.all` as we did with the previous example
* A section may be partitioned into one or more segments. Each section will have the same attributes and inserted mechanisms. However, the actual property values of each individual segment, like distance and conductance, may vary. Hence, we need to update each individual segment of each section using the code:
```python
    for section in hoc_obj.all:
        for segment in section:
            ....
```
* [`h.distance(seg)`](https://www.neuron.yale.edu/neuron/static/new_doc/modelspec/programmatic/topology/geometry.html#distance) is a built-in NEURON function that measures arc-length from each segment to the soma, in microns. Only segments that are >1000 um will have their Ih conductances reduced. The rest will be left to default value.
* We use the built-in Python functions `getattr` and `setattr` to fetch and set the gbar_Ih value of each segment object.


In [6]:
bionet.reset()
conf = bionet.Config.from_json('config.modified_densities.json')
conf.build_env()

graph = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=graph)
sim.run()

2024-05-01 17:51:56,039 [INFO] Created log file
Mechanisms already loaded from path: ./components/mechanisms.  Aborting.
2024-05-01 17:51:56,073 [INFO] Building cells.
2024-05-01 17:51:57,604 [INFO] 262 out of 573 apical dendritic segments updated.
2024-05-01 17:51:57,609 [INFO] Building recurrent connections
2024-05-01 17:51:57,616 [INFO] Building virtual cell stimulations for virt_exc_spikes
2024-05-01 17:51:57,664 [INFO] Running simulation for 2000.000 ms with the time step 0.100 ms
2024-05-01 17:51:57,665 [INFO] Starting timestep: 0 at t_sim: 0.000 ms
2024-05-01 17:51:57,666 [INFO] Block save every 5000 steps
2024-05-01 17:52:04,210 [INFO]     step:5000 t_sim:500.00 ms
2024-05-01 17:52:10,637 [INFO]     step:10000 t_sim:1000.00 ms
2024-05-01 17:52:16,925 [INFO]     step:15000 t_sim:1500.00 ms
2024-05-01 17:52:23,295 [INFO]     step:20000 t_sim:2000.00 ms
2024-05-01 17:52:23,309 [INFO] Simulation completed in 25.65 seconds 


#### Help with investigating and updating a model

Trying to figure out what mechanisms, section types, and other parameters of instantiated HOC cell can be tricky, as can trying to figure out how to change an existing cell. A good place to start is by looking through the official NEURON documentation [1](https://www.neuron.yale.edu/neuron/static/new_doc/modelspec/programmatic/topology/geometry.html) [2](https://www.neuron.yale.edu/neuron/static/new_doc/modelspec/programmatic/topology/geometry.html). One useful function is the `h.psection()` that will provide valuable information about each section/segment of a cell.

Below, we can overwrite the previous version of `adjust_densities` with one that will print information about each section type (although only for the first section of each type - there are 1200+ sections in total, so we won't try to print them all out). Not only is this helpful for inspecting the cell, it shows how useful it can be to change the way cells are processed on the fly.


In [11]:
from pprint import pprint

@bionet.model_processing(name='adjust_densities')
def check_mechanisms(hoc_obj, cell, dynamics_params):
    for sec in hoc_obj.apic:
        print(sec.name())
        pprint(sec.psection())
        break

    return hoc_obj

bionet.reset()
conf = bionet.Config.from_json('config.modified_densities.json')
conf.build_env()

graph = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=graph)

2024-05-01 18:05:36,286 [INFO] Created log file
Mechanisms already loaded from path: ./components/mechanisms.  Aborting.
2024-05-01 18:05:36,316 [INFO] Building cells.
L5PCtemplate[5].apic[0]
{'Ra': 100.0,
 'cell': L5PCtemplate[5],
 'cm': [2.0],
 'density_mechs': {'CaDynamics_E2': {'decay': [35.725651],
                                     'depth': [0.1],
                                     'gamma': [0.000637],
                                     'minCai': [0.0001]},
                   'Ca_HVA': {'g': [0.0],
                              'gbar': [1e-05],
                              'h': [0.0],
                              'ica': [0.0],
                              'm': [0.0]},
                   'Ca_LVAst': {'gCa_LVAst': [0.0],
                                'gCa_LVAstbar': [0.00141954],
                                'h': [0.0],
                                'ica': [0.0],
                                'm': [0.0]},
                   'Ih': {'g': [0.0],
                     